In [1]:
import cv2
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets

In [4]:
# Load the video
video = cv2.VideoCapture('./media/run1.mp4')

# Load the face cascade classifier
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# Load the face detector model
prototxt_path = './models/deploy.prototxt'
caffemodel_path = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detector = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Initialize the total number of faces detected
total_faces = 0

# Create a video widget
video_widget = widgets.Output()

# Display the video widget
display(video_widget)

Output()

In [5]:
# Initialize the total number of faces detected
total_faces = 0

# Create a video widget
video_widget = widgets.Output()

# Display the video widget
display(video_widget)


Output()

In [6]:
# Loop through each frame of the video
while True:
    # Read the frame
    ret, frame = video.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Resize the frame for faster processing
    frame_resized = cv2.resize(frame, (300, 300))

    # Convert the frame to a blob for input to the face detector
    blob = cv2.dnn.blobFromImage(frame_resized, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Pass the blob through the face detector
    face_detector.setInput(blob)
    detections = face_detector.forward()

    # Loop through each face detection
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Ignore detections with low confidence
        if confidence < 0.5:
            continue

        # Get the coordinates of the face detection
        box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
        (x, y, w, h) = box.astype('int')

        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Count the number of faces in the frame
    num_faces = detections.shape[2]

    # Add the number of faces to the total count
    total_faces += num_faces

    # Display the total number of faces detected in the frame
    cv2.putText(frame, f'Total Faces: {total_faces}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame with detected rectangles in the video widget
    with video_widget:
        clear_output(wait=True)
        _, png = cv2.imencode('.png', frame[:, :, ::-1])
        display(widgets.Image(value=png.tobytes()))

error: OpenCV(4.7.0) /io/opencv/modules/dnn/src/layers/batch_norm_layer.cpp:51: error: (-215:Assertion failed) blobs.size() >= 2 in function 'BatchNormLayerImpl'


In [ ]:
# Release the video capture object
video.release()

# Display the total count of faces detected
print('Total faces:', total_faces)